## Preparing and cleaning data up for classification

In [1]:
import pandas as pd
import numpy as np

## read csv to dataframe
fb_df = pd.read_csv("fbcleaned.csv")
insta_df = pd.read_csv("instacleaned.csv")
print(fb_df)
print(insta_df)

   FacebookTime        Day           Date  Year
0        6:57pm   Saturday     October 28  2020
1        1:12pm     Sunday    October 25   2020
2        4:10pm    Tuesday     October 13  2020
3        5:34pm  Wednesday   September 30  2020
4        6:34pm     Monday   September 28  2020
..          ...        ...            ...   ...
62      11:10am     Monday   September 30  2019
63       8:27am     Monday   September 23  2029
64       1:10pm   Thursday   September 19  2019
65       3:30pm     Monday      August 19  2019
66       7:27pm   Saturday      August 03  2019

[67 rows x 4 columns]
    InstagramTime        Day           Date  Year
0          7:30pm  Wednesday   September 30  2020
1          8:28pm   Saturday   September 26  2020
2          2:05pm   Saturday   September 12  2020
3          9:56pm     Friday   September 11  2020
4          6:44pm     Friday   September 11  2020
..            ...        ...            ...   ...
63         8:46pm     Friday     October 18  2019
6

In [2]:
### Days will be change to numeric values, Monday is 0, Tuesday is 1, wednesday is 2 and so on
###Times before 5:00pm (from 7:00am to 4:59pm) will be classify as the group "before5" and times after 5:00pm (from 5:00pm to 12:59am) is "after5"

# Let's start by changing for both social media data labels
i =0
j = 0

fb_dfclass = fb_df
fb_dfclass.rename(columns={'FacebookTime':'Time','Day':'Day','Date':'Date','Year':'Year'},inplace=True)
fb_dfclass["Social Media"] = np.zeros(len(fb_dfclass))
insta_dfclass = insta_df
#insta_dfclass.rename(columns={'InstagramTime':'Time','Day':'Day','Date':'Date','Year':'Year'},inplace=True)
insta_dfclass.columns.values[0] = "Time"
insta_dfclass["Social Media"] = np.ones(len(insta_dfclass))
s_dfclass = pd.concat([fb_dfclass,insta_dfclass],axis= 0,ignore_index=True)
s_dfclass.reset_index(drop=True)
j =0
for index,rows in s_dfclass.iterrows():
    fbtime = rows[0]
    if(fbtime.find("am")!= -1):
        s_dfclass.iloc[j,0] = "before5"
    elif(fbtime.find("pm")!= -1): #it can be before 5:00pm like 1:00,2:00,3:00pm etc. so we have to compare
        if(int(fbtime.split(':')[0]) < 5):
            s_dfclass.iloc[j,0] = "before5" 
        else:
            s_dfclass.iloc[j,0] = "after5"
    j+=1
    
#split date to Month and Date
s_dfclass[['Month','Date']] = s_dfclass.Date.str.split(expand=True) 


        
print(s_dfclass)
s_dfclass.to_csv("cleanedSM.csv",index=False)


        Time        Day Date  Year  Social Media      Month
0     after5   Saturday   28  2020           0.0    October
1    before5     Sunday   25  2020           0.0    October
2    before5    Tuesday   13  2020           0.0    October
3     after5  Wednesday   30  2020           0.0  September
4     after5     Monday   28  2020           0.0  September
..       ...        ...  ...   ...           ...        ...
130   after5     Friday   18  2019           1.0    October
131  before5    Tuesday   01  2019           1.0    October
132  before5   Thursday   19  2019           1.0  September
133  before5     Monday   19  2019           1.0     August
134   after5   Saturday   03  2019           1.0     August

[135 rows x 6 columns]


In [3]:
s_dfclass = pd.read_csv("cleanedSM.csv")
print(s_dfclass)

days = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
s_days = s_dfclass.groupby("Day")
before_5 =[]
after_5 =[]
fbday_freq = []

for day in days :
    s_df = s_days.get_group(day)
    time_df = s_dfclass.groupby("Time")
    #print(day_df)
    after_5.append(len(time_df.get_group("after5")))
    before_5.append(len(time_df.get_group("before5")))

        Time        Day  Date  Year  Social Media      Month
0     after5   Saturday    28  2020           0.0    October
1    before5     Sunday    25  2020           0.0    October
2    before5    Tuesday    13  2020           0.0    October
3     after5  Wednesday    30  2020           0.0  September
4     after5     Monday    28  2020           0.0  September
..       ...        ...   ...   ...           ...        ...
130   after5     Friday    18  2019           1.0    October
131  before5    Tuesday     1  2019           1.0    October
132  before5   Thursday    19  2019           1.0  September
133  before5     Monday    19  2019           1.0     August
134   after5   Saturday     3  2019           1.0     August

[135 rows x 6 columns]


In [8]:
from sklearn import preprocessing

y = s_dfclass["Time"]
X = s_dfclass.drop("Time", axis=1)
X = X.drop("Date",axis =1)
X = X.drop("Year",axis=1)
#print(X)
#print(y)

##Convert data to be numerial
le = preprocessing.LabelEncoder()
le.fit(X["Social Media"])
X["Social Media"] = le.transform(X["Social Media"])
le.fit(X["Day"])
X["Day"] = le.transform(X["Day"])
le.fit(X["Month"]) 
X["Month"]= le.transform(X["Month"])
print(X)

     Day  Social Media  Month
0      2             0     11
1      3             0     11
2      5             0     11
3      6             0     12
4      1             0     12
..   ...           ...    ...
130    0             1     11
131    5             1     11
132    4             1     12
133    1             1      1
134    2             1      1

[135 rows x 3 columns]


In [11]:
from sklearn import preprocessing

y = s_dfclass["Time"]
X = s_dfclass.drop("Time", axis=1)
X = X.drop("Date",axis =1)
X = X.drop("Year",axis=1)

##Convert data to be numerial
le = preprocessing.LabelEncoder()
le.fit(X["Social Media"])
X["Social Media"] = le.transform(X["Social Media"])
le.fit(X["Day"])
X["Day"] = le.transform(X["Day"])
le.fit(X["Month"])
X["Month"] = le.transform(X["Month"])
print(X)

     Day  Social Media  Month
0      2             0     11
1      3             0     11
2      5             0     11
3      6             0     12
4      1             0     12
..   ...           ...    ...
130    0             1     11
131    5             1     11
132    4             1     12
133    1             1      1
134    2             1      1

[135 rows x 3 columns]


## KNN Neighbors classifier

In [12]:

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier


X_train,X_test, y_train, y_test = train_test_split(X,y,random_state = 0,stratify=y)# random state for reproducility so that we will always get same data
clf = KNeighborsClassifier(n_neighbors=20, metric="euclidean")
clf.fit(X_train, y_train)

y_predicted = clf.predict(X_test)
accuracy = clf.score(X_test, y_test)
print(accuracy)


0.5882352941176471


## Decision Tree Classifier

In [13]:
from sklearn.tree import DecisionTreeClassifier

tree_clf = DecisionTreeClassifier(random_state=0,max_depth=3)
tree_clf.fit(X_train, y_train)
y_predicted_tree = tree_clf.predict(X_test)
accuracy_tree = tree_clf.score(X_test, y_test)
print(accuracy_tree)

0.5588235294117647


In [25]:
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import accuracy_score

for model in [clf, tree_clf]:
    print(type(model))
    accuracies = cross_val_score(model, X, y, cv=4)
    print(accuracies, accuracies.mean())
    y_predictions = cross_val_predict(model, X, y, cv=4) 
    # better estimate of accuracy
    accuracy = accuracy_score(y, y_predictions)
    print(accuracy)

<class 'sklearn.neighbors._classification.KNeighborsClassifier'>
[0.55882353 0.52941176 0.5        0.57575758] 0.5409982174688057
0.5407407407407407
<class 'sklearn.tree._classes.DecisionTreeClassifier'>
[0.61764706 0.47058824 0.44117647 0.60606061] 0.5338680926916222
0.5333333333333333
